In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [3]:
# Set random seeds for reproducibility
torch.manual_seed(42)

In [4]:
# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [5]:
df=pd.read_csv('fmnist_small.csv')

In [6]:
X=df.drop('label',axis=1)
y=df['label']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# We have to transform our data to fit in pre trained model

# transformations
from torchvision.transforms import transforms

custom_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [42]:
from torch.utils.data import Dataset
from PIL import Image
import numpy as np
import torch

class CustomDataset(Dataset):
    def __init__(self, features, labels, transform):
        self.features = features  # likely a DataFrame
        self.labels = labels      # likely a Series or DataFrame column
        self.transform = transform

    def __len__(self):
        return len(self.features)

    def __getitem__(self, index):
        # Get the row and reshape to 28x28
        image = self.features.iloc[index].values.reshape(28, 28).astype(np.uint8)

        # Convert grayscale to RGB (3-channel)
        image = np.stack([image] * 3, axis=-1)

        # Convert to PIL Image
        image = Image.fromarray(image)

        # Apply transform
        image = self.transform(image)

        # Get label
        label = torch.tensor(self.labels.iloc[index], dtype=torch.long)

        return image, label


In [44]:
train_dataset = CustomDataset(X_train, y_train, transform=custom_transform)
test_dataset = CustomDataset(X_test, y_test, transform=custom_transform)

In [45]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [46]:
# fetch the pretrained model

import torchvision.models as models

vgg16 = models.vgg16(pretrained=True)

In [47]:
vgg16

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [48]:
# Freezing feature layers parameters
for param in vgg16.features.parameters():
  param.requires_grad=False

In [49]:
vgg16.classifier = nn.Sequential(
    nn.Linear(25088, 1024),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, 10)
)

In [50]:
vgg16 = vgg16.to(device)

In [54]:
learning_rate = 0.0001
epochs = 100

In [55]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg16.classifier.parameters(), lr=learning_rate)

In [56]:
# training loop

for epoch in range(epochs):

  total_epoch_loss = 0

  for batch_features, batch_labels in train_loader:

    # move data to gpu
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    # forward pass
    outputs = vgg16(batch_features)

    print(outputs.shape)
    print(batch_labels.shape)

    # calculate loss
    loss = criterion(outputs, batch_labels)

    # back pass
    optimizer.zero_grad()
    loss.backward()

    # update grads
    optimizer.step()

    total_epoch_loss = total_epoch_loss + loss.item()

    break

  avg_loss = total_epoch_loss/len(train_loader)
  print(f'Epoch: {epoch + 1} , Loss: {avg_loss}')



torch.Size([32, 10])
torch.Size([32])
Epoch: 1 , Loss: 0.012084612051645915
torch.Size([32, 10])
torch.Size([32])
Epoch: 2 , Loss: 0.010795138676961262
torch.Size([32, 10])
torch.Size([32])
Epoch: 3 , Loss: 0.012203238010406493
torch.Size([32, 10])
torch.Size([32])
Epoch: 4 , Loss: 0.010880751609802246
torch.Size([32, 10])
torch.Size([32])
Epoch: 5 , Loss: 0.011050163904825846
torch.Size([32, 10])
torch.Size([32])
Epoch: 6 , Loss: 0.009534938335418701
torch.Size([32, 10])
torch.Size([32])
Epoch: 7 , Loss: 0.009087337652842203
torch.Size([32, 10])
torch.Size([32])
Epoch: 8 , Loss: 0.00897388219833374
torch.Size([32, 10])
torch.Size([32])
Epoch: 9 , Loss: 0.008076705137888591
torch.Size([32, 10])
torch.Size([32])
Epoch: 10 , Loss: 0.007894374529520671
torch.Size([32, 10])
torch.Size([32])
Epoch: 11 , Loss: 0.008121825059254965
torch.Size([32, 10])
torch.Size([32])
Epoch: 12 , Loss: 0.007643632888793946
torch.Size([32, 10])
torch.Size([32])
Epoch: 13 , Loss: 0.008113433519999186
torch.Siz

In [57]:
# evaluation on test data
total = 0
correct = 0

with torch.no_grad():

  for batch_features, batch_labels in test_loader:

    # move data to gpu
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    outputs = vgg16(batch_features)

    _, predicted = torch.max(outputs, 1)

    total = total + batch_labels.shape[0]

    correct = correct + (predicted == batch_labels).sum().item()

print(correct/total)

0.8091666666666667


In [58]:
# evaluation on training data
total = 0
correct = 0

with torch.no_grad():

  for batch_features, batch_labels in train_loader:

    # move data to gpu
    batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

    outputs = vgg16(batch_features)

    _, predicted = torch.max(outputs, 1)

    total = total + batch_labels.shape[0]

    correct = correct + (predicted == batch_labels).sum().item()

print(correct/total)

0.848125
